# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
##############################
#                            #
#  Homework 6 - Python API   #
#  Student - Matheus Gratz   #
#  Part II - Vacation PY     #
##############################

In [2]:
# Dependencies and Setup
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#Set filepath
file = 'output_data/weather_data.csv'

#Read and store CSV to DF
weather_df = pd.read_csv(file)
weather_df.dropna(inplace=True)

#Show DataFrame
weather_df

,City ID,City Name,Country,Latitude,Longitude,Hemisphere,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,3366880,hermanus,ZA,-34.42,19.23,South,55.00,95,30,5.99
1,6185377,yellowknife,CA,62.46,-114.35,North,50.41,57,20,6.93
2,1507390,dikson,RU,73.51,80.55,North,41.14,84,1,10.22
3,5557293,sitka,US,57.05,-135.33,North,53.60,93,90,3.36
4,5106841,albany,US,42.60,-73.97,North,72.55,91,100,4.59
...,...,...,...,...,...,...,...,...,...,...
542,3424901,aasiaat,GL,68.71,-52.87,North,30.81,98,85,4.34
543,2063042,port hedland,AU,-20.32,118.57,South,89.60,32,32,11.41
544,2156927,moree,AU,-29.47,149.85,South,77.00,33,0,14.99
545,1684606,taganak,PH,6.08,118.30,North,87.40,66,40,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [5]:
# Plot Heatmap
locations = weather_df[['Latitude', 'Longitude']]
humidity = weather_df['Humidity (%)']

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=humidity,
max_intensity = 200, point_radius=30, opacity = 0.50)
markers = gmaps.marker_layer(locations)

#Add Layer and show map
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Create datafrane
ideal_weather_df = weather_df[
    (weather_df['Temperature (F)']> 70) & 
    (weather_df['Temperature (F)']<80) &
    (weather_df['Cloudiness (%)']==0)
]

#checking how many cities filtered to not blow up requests on google maps
ideal_weather_df.shape

(27, 10)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Set DF to receive data
hotel_df = ideal_weather_df[['City Name', 'Country', 'Latitude', 'Longitude']]
hotel_df.reset_index(inplace = True)
hotel_df = hotel_df[['City Name', 'Country', 'Latitude', 'Longitude']]
hotel_df['Hotel Name'] = ""
hotel_df['Hotel Lat'] = ""
hotel_df['Hotel Long'] = ""
hotel_df.head()

,City Name,Country,Latitude,Longitude,Hotel Name,Hotel Lat,Hotel Long
0,jiuquan,CN,39.74,98.52,,,
1,lodwar,KE,3.12,35.60,,,
2,gazanjyk,TM,39.24,55.52,,,
3,gat,IL,31.61,34.76,,,
4,tiznit,MA,29.58,-9.50,,,


In [8]:
#Set Base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set Paramaters
params = {
    "type": "lodging",
    "radius": 5000,
    "key": g_key,
    }

#Set some variables to help creating logs, etc.
count_ok = 0
count_nok = 0
total_hotel = len(hotel_df["City Name"])

#Loop & Request
for index, row in hotel_df.iterrows():
        
    loc = str(row['Latitude']) + ', ' + str(row['Longitude'])
    params['location'] = loc
    response = requests.get(base_url, params=params).json()

    results = response['results']
    
    clear_output(wait=True) 
    try:
        print(f'Iteration {count_ok + count_nok +1} of {total_hotel}:')
        print(f"Closest Hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Hotel Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Hotel Long'] = results[0]['geometry']['location']['lng']
        count_ok += 1
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df.loc[index, 'Hotel Name'] = np.nan
        count_nok += 1
       
print("------------")
print('We are Done!')
print(f"We've searched for {total_hotel} hotels and we've got \n{count_ok} successful responses and \n{count_nok} unsuccessful responses")
print("------------")

Iteration 27 of 27:
Closest Hotel is Molika Springs Motel.
------------
We are Done!
We've searched for 27 hotels and we've got 
23 successful responses and 
4 unsuccessful responses
------------


In [9]:
#Drop non successful requests
hotel_df.dropna(inplace = True)
hotel_df

,City Name,Country,Latitude,Longitude,Hotel Name,Hotel Lat,Hotel Long
0,jiuquan,CN,39.74,98.52,Jiuquan Hotel,39.7374,98.5078
1,lodwar,KE,3.12,35.60,Turkwel Lodge,3.11554,35.6041
2,gazanjyk,TM,39.24,55.52,Railway motel,39.2632,55.4896
3,gat,IL,31.61,34.76,OlusHome,31.6131,34.716
5,guerrero negro,MX,27.98,-114.06,"Casa Laguna, Bed & Breakfast",27.9593,-114.055
6,cayenne,GF,4.93,-52.33,Hôtel Le Dronmi,4.9388,-52.3314
7,arraial do cabo,BR,-22.97,-42.03,Pousada Porto Praia,-22.9777,-42.0239
9,sukhumi,GE,43.00,41.02,Ritsa Hotel,43.0008,41.0237
10,salinopolis,BR,-0.61,-47.36,Hotel Salinópolis,-0.612602,-47.3586
11,suzun,RU,53.78,82.31,Elegant Lyuks,53.7783,82.3227


In [10]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<p style="color:black;"><dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl></p>
"""

In [11]:
# Add marker layer ontop of heat map
heat_loc = weather_df[['Latitude', 'Longitude']]
heat_hum = weather_df['Humidity (%)']

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[['Hotel Lat', 'Hotel Long']]

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    heat_loc, weights=heat_hum,
max_intensity = 200, point_radius=30, opacity = 0.50)
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(heatmap_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))